In [1]:
import pandas as pd
import numpy as np

In [26]:
allProjects = pd.read_csv('/Users/francescamorini/projects/water-data/python/src/results/microdata/microdata.csv', sep=',')

In [27]:
allProjects.head(2)

,Unnamed: 0,DonorName,RecipientName,DonorCode,RecipientCode,IncomegroupName,USD_Commitment_Defl,USD_Received_Defl,ShortDescription,AgencyName,...,Interest1,Interest2,Repaydate1,Repaydate2,USD_Interest,Finance_t,Recipientstat SI.SPR.PCAP,Recipientstat SI.POV.XPND.MD,Recipientstat SP.POP.TOTL,Recipientstat AG.SRF.TOTL.K2
0,300,Germany,Niger,5,260,LDCs,23.3681,NaN,TOWN WATER SUPPLY,Kreditanstalt für Wiederaufbau,...,750.0,0.0,1983-12-01,2023-12-01,NaN,421,NaN,NaN,NaN,NaN
1,325,Germany,Tanzania,5,282,LDCs,18.5488,NaN,RURAL WATER SUPPLY,Kreditanstalt für Wiederaufbau,...,750.0,0.0,1983-12-01,2023-06-01,NaN,421,NaN,NaN,NaN,NaN


In [56]:
allProjects.columns

Index(['Unnamed: 0', 'DonorName', 'RecipientName', 'DonorCode',
       'RecipientCode', 'IncomegroupName', 'USD_Commitment_Defl',
       'USD_Received_Defl', 'ShortDescription', 'AgencyName', 'FlowCode',
       'SectorCode', 'ProjectTitle', 'PurposeName', 'SectorName',
       'ChannelName', 'ChannelReportedName', 'ExpectedStartDate',
       'CompletionDate', 'LongDescription', 'CommitmentDate', 'USD_GrantEquiv',
       'TypeRepayment', 'NumberRepayment', 'Interest1', 'Interest2',
       'Repaydate1', 'Repaydate2', 'USD_Interest', 'Finance_t',
       'Recipientstat SI.SPR.PCAP', 'Recipientstat SI.POV.XPND.MD',
       'Recipientstat SP.POP.TOTL', 'Recipientstat AG.SRF.TOTL.K2',
       'CommitmentYear'],
      dtype='object')

Getting mean, max and min commitments across all projects.

In [30]:
CommitmentMean = allProjects['USD_Commitment_Defl'].mean()
CommitmentMedian = allProjects['USD_Commitment_Defl'].median()
CommitmentMin = allProjects['USD_Commitment_Defl'].min()
CommitmentMax = allProjects['USD_Commitment_Defl'].max()
print(CommitmentMean, CommitmentMedian, CommitmentMin, CommitmentMax)

4.278813743241011 0.2155805 6.63999e-05 254.08


In [32]:
allProjects['CommitmentYear'] = pd.DatetimeIndex(allProjects['CommitmentDate']).year
allProjects = allProjects[allProjects['CommitmentYear'].notna()]
allProjects['CommitmentYear'] = allProjects['CommitmentYear'].astype(int)

<ipython-input-32-75aff76b8d14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allProjects['CommitmentYear'] = allProjects['CommitmentYear'].astype(int)


In [33]:
yearsList = allProjects['CommitmentYear'].unique()
print(yearsList)

[1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986
 1987 1988 1989 1990 1991 1992 1993 1994 1997 1999 1995 1996 1998 2000
 2001 2003 2002 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017 2018 2019]


In [35]:
for year in yearsList:
    yearProjects = allProjects[allProjects['CommitmentYear'] == year]
    projectsNumber = yearProjects['USD_Commitment_Defl'].count()
    percAbove = (yearProjects.loc[yearProjects['USD_Commitment_Defl'] > CommitmentMedian,'USD_Commitment_Defl'].count() 
    / projectsNumber) * 100
    percBelow = (yearProjects.loc[yearProjects['USD_Commitment_Defl'] < CommitmentMedian,'USD_Commitment_Defl'].count() 
    / projectsNumber) * 100
    print(year)
    print('above ', percAbove)
    print('below ', percBelow)
    print('number of projects ', projectsNumber)
    print('-----')

1973
above  100.0
below  0.0
number of projects  2
-----
1974
above  100.0
below  0.0
number of projects  10
-----
1975
above  100.0
below  0.0
number of projects  11
-----
1976
above  100.0
below  0.0
number of projects  16
-----
1977
above  100.0
below  0.0
number of projects  17
-----
1978
above  100.0
below  0.0
number of projects  9
-----
1979
above  93.33333333333333
below  6.666666666666667
number of projects  15
-----
1980
above  100.0
below  0.0
number of projects  16
-----
1981
above  100.0
below  0.0
number of projects  16
-----
1982
above  100.0
below  0.0
number of projects  21
-----
1983
above  95.83333333333334
below  4.166666666666666
number of projects  24
-----
1984
above  100.0
below  0.0
number of projects  20
-----
1985
above  100.0
below  0.0
number of projects  27
-----
1986
above  96.15384615384616
below  3.8461538461538463
number of projects  26
-----
1987
above  100.0
below  0.0
number of projects  22
-----
1988
above  96.96969696969697
below  3.03030303030303

Here I calculate how the total is distributed over projects for one year. This helps in understanding the gradual change of strategies. From few projects in few countries to a variety of smaller projects allocated in many more countries.

In [62]:
yearlyPercentages = []
for year in yearsList:
    yearProjects = allProjects[allProjects['CommitmentYear'] == year]
    yearlySum = yearProjects['USD_Commitment_Defl'].sum()
    for index, row in yearProjects.iterrows():
        # progetto * 100 / totale
        projectPerc = row['USD_Commitment_Defl'] * 100 / yearlySum
        yearlyPercentages.append(
            [
                row['RecipientName'], 
                row['AgencyName'], 
                row['ProjectTitle'], 
                row['USD_Commitment_Defl'], 
                yearlySum, 
                projectPerc,
                year
            ]
        )

yearlyProjectsPercentages = pd.DataFrame.from_records(yearlyPercentages, columns=['RecipientName', 'AgencyName', 'ProjectTitle', 'USD_Commitment_Defl', 'yearly_USD_Commitment_Defl', 'project_Perc_onYear', 'Commitment_year'])


In [61]:
yearlyProjectsPercentages.to_csv('data/HistoricalChange/percentages_on_yearly_total.csv')